In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import networkx as nx
import warnings

In [2]:
import gc
import pickle

In [3]:
from tqdm.notebook import tqdm

In [4]:
def load_file(file):

    r = open(file,'rb')
    x = pickle.load(r)
    r.close()

    return x

In [5]:
def save_file(x,file):

    r = open(file,'wb')
    pickle.dump(x,r)

In [6]:
warnings.filterwarnings('ignore')

In [7]:
F = load_file('XXX.pkl')

In [8]:
adjacency_listT = pd.DataFrame(F.edges())

In [9]:
adjacency_listT.columns = ['Theorem_id','Ref_id']

In [10]:
import random

In [11]:
FTheorem1 = F.reverse(copy=True)

In [12]:
topological_generationsTheorem = [sorted(generation) for generation in nx.topological_generations(FTheorem1)]

In [13]:
topological_sortTheorem = []

In [14]:
for i in range(len(topological_generationsTheorem)):
    topological_generations1 = topological_generationsTheorem[i]
    random.shuffle(topological_generations1)
    topological_sortTheorem.extend(topological_generations1)

In [15]:
topological_sortdict = dict(zip(topological_sortTheorem,range(0,len(topological_sortTheorem),1)))

In [16]:
list1 = adjacency_listT[['Theorem_id']]
list2 = adjacency_listT[['Ref_id']]
list1['topological_sort1'] = list1['Theorem_id'].map(topological_sortdict)
list2['topological_sort2'] = list2['Ref_id'].map(topological_sortdict)
list1 = list1.sort_values(by=['topological_sort1'], ascending=True)
list2 = list2.sort_values(by=['topological_sort2'], ascending=True)

In [17]:
list1['no'] = pd.factorize(list1['Theorem_id'])[0]
list1.set_index(['no'], inplace=True)
list2.index = np.arange(len(list2.index))

In [18]:
newadjacency_list = []

In [19]:
lenth = np.max(list1['no'])+1

In [20]:
for i in tqdm(range(lenth)):
    list1i = list1.loc[[i]]
    list1jpr = list2[list2['topological_sort2']<list(list1i['topological_sort1'])[0]]
    list1j = list1jpr.sample(n=len(list1i))
    list2 = list2[~list2.index.isin(list1j.index)]
    #list1 = list1[~list1.index.isin(list1i.index)]
    listnew = pd.concat([list1i.reset_index(drop=True),list1j.reset_index(drop=True)],axis=1)
    newadjacency_list.extend(listnew.values.tolist())

In [21]:
newadjacency_list1 = pd.DataFrame(newadjacency_list, columns=['Theorem_id','topological_sort1','Ref_id','topological_sort2'])

In [22]:
FTheoremRN = nx.from_pandas_edgelist(newadjacency_list1, 'Theorem_id', 'Ref_id', create_using=nx.DiGraph())

In [23]:
save_file(FTheoremRN,'XXX.pkl')